In [1]:
import math
import random
import time
from tkinter import *
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from statistics import mode

######################################################################
# This section contains functions for clustering a dataset
# using the k-means algorithm.
######################################################################

def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

def computeDistance(x, y, similarity='Euclidean'):
    if x == None or y == None:
        return float("inf")
    distance = 0
    if similarity == 'Euclidean':
        for i in range(1, len(x)):
            distance += (x[i]-y[i])**2
        return distance
    elif similarity == 'Manhattan':
        for i in range(1, len(x)):
            distance += abs(x[i]-y[i])
        return distance
    elif similarity == 'Jaccard':
        return 1 - float(len(set(x).intersection(set(y)))) / len(set(x).union(set(y)))         
    elif similarity == 'Cosine':
        newInstance1 = np.array(x[1:]).reshape(1,-1)
        newInstance2 = np.array(y[1:]).reshape(1,-1)
        return 1 - cosine_similarity(newInstance1, newInstance2)

def assign(instance, centroids, similarity='Euclidean'):
    minDistance = computeDistance(instance, centroids[0], similarity)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = computeDistance(instance, centroids[i], similarity)
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def assignAll(instances, centroids, similarity='Euclidean'):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids, similarity)
        clusters[clusterIndex].append(instance)
    return clusters

def computeWithinss(clusters, centroids, similarity='Euclidean'):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += computeDistance(centroid, instance, similarity)
    return result

def sumOfSquares(clusters, centroids):
    sse = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            for i in range(1, len(centroid)):
                sse += (centroid[i]-instance[i])**2
    return round(sse, 3)

def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    means = [name] + [0] * (numAttributes-1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            means[i] += instance[i]
    for i in range(1, numAttributes):
        means[i] /= float(numInstances)
    return tuple(means)

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        #name = "centroid" + str(i)
        name = i
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

def kmeans(instances, canvas, k, animation=False, similarity='Euclidean', initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        # randomly select k initial centroids
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    if animation:
        delay = 1.0 # seconds
        clusters = createEmptyListOfLists(k)
        clusters[0] = instances
        paintClusters2D(canvas, clusters, centroids, "Initial centroids")
        time.sleep(delay)
    iteration = 0
    while (centroids != prevCentroids):
        iteration += 1
        clusters = assignAll(instances, centroids, similarity)
        if animation:
            paintClusters2D(canvas, clusters, centroids, "Assign %d" % iteration)
            time.sleep(delay)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, similarity)
        if animation:
            paintClusters2D(canvas, clusters, centroids,
                            "Update %d, withinss %.1f" % (iteration, withinss))
            time.sleep(delay)
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["similarity"] = similarity
    result["SSE"] = sumOfSquares(clusters, centroids)
    result["iterations"] = iteration
    return result

# Repeats k-means clustering n times, and returns the clustering
# with the smallest withinss
def repeatedKMeans(instances, k, n):
    bestClustering = {}
    bestClustering["withinss"] = float("inf")
    for i in range(1, n+1):
        print ("k-means trial %d," % i)
        trialClustering = kmeans(instances, k)
        print ("withinss: %.1f" % trialClustering["withinss"])
        if trialClustering["withinss"] < bestClustering["withinss"]:
            bestClustering = trialClustering
            minWithinssTrial = i
    print ("Trial with minimum withinss:", minWithinssTrial)
    return bestClustering

######################################################################
# This section contains functions for visualizing datasets and
# clustered datasets.
######################################################################

def printTable(instances):
    for instance in instances:
        if instance != None:
            line = instance[0] + "\t"
            for i in range(1, len(instance)):
                line += "%.2f " % instance[i]
            print (line)

def drawPoints(canvas, instances, color, shape):
    random.seed(0)
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for instance in instances:
        x = 5*(random.random()-0.5)+margin+(instance[1]-minX)*scaleX
        y = 5*(random.random()-0.5)+height-margin-(instance[2]-minY)*scaleY
        if (shape == "square"):
            canvas.create_rectangle(x-5, y-5, x+5, y+5, fill=color)
        else:
            canvas.create_oval(x-5, y-5, x+5, y+5, outline=color)
    canvas.update()

def connectPoints(canvas, instances1, instances2, color):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    scaleX = float(width - 2*margin) / (maxX - minX)
    scaleY = float(height - 2*margin) / (maxY - minY)
    for p1 in instances1:
        for p2 in instances2:
            x1 = margin + (p1[1]-minX)*scaleX
            y1 = height - margin - (p1[2]-minY)*scaleY
            x2 = margin + (p2[1]-minX)*scaleX
            y2 = height - margin - (p2[2]-minY)*scaleY
            canvas.create_line(x1, y1, x2, y2, fill=color)
    canvas.update()

def mergeClusters(clusters):
    result = []
    for cluster in clusters:
        result.extend(cluster)
    return result

def paintAxes(canvas):
    width = canvas.winfo_reqwidth()
    height = canvas.winfo_reqheight()
    margin = canvas.data["margin"]
    minX = canvas.data["minX"]
    minY = canvas.data["minY"]
    maxX = canvas.data["maxX"]
    maxY = canvas.data["maxY"]
    canvas.create_line(margin/2, height-margin/2, width-5, height-margin/2,
                       width=2, arrow=LAST)
    canvas.create_text(margin, height-margin/4,
                       text=str(minX), font="Sans 11")
    canvas.create_text(width-margin, height-margin/4,
                       text=str(maxX), font="Sans 11")
    canvas.create_line(margin/2, height-margin/2, margin/2, 5,
                       width=2, arrow=LAST)
    canvas.create_text(margin/4, height-margin,
                       text=str(minY), font="Sans 11", anchor=W)
    canvas.create_text(margin/4, margin,
                       text=str(maxY), font="Sans 11", anchor=W)
    canvas.update()

def showDataset2D(canvas, instances):
    canvas.delete(ALL)
    paintAxes(canvas)
    drawPoints(canvas, instances, "blue", "circle")
    canvas.update()

def showClusters2D(clusteringDictionary):
    clusters = clusteringDictionary["clusters"]
    centroids = clusteringDictionary["centroids"]
    withinss = clusteringDictionary["withinss"]
    canvas = prepareWindow(mergeClusters(clusters))
    paintClusters2D(canvas, clusters, centroids,
                    "Withinss: %.1f" % withinss)

def paintClusters2D(canvas, clusters, centroids, title=""):
    canvas.delete(ALL)
    paintAxes(canvas)
    colors = ["blue", "red", "green", "brown", "purple", "orange"]
    for clusterIndex in range(len(clusters)):
        color = colors[clusterIndex%len(colors)]
        instances = clusters[clusterIndex]
        centroid = centroids[clusterIndex]
        drawPoints(canvas, instances, color, "circle")
        if (centroid != None):
            drawPoints(canvas, [centroid], color, "square")
        connectPoints(canvas, [centroid], instances, color)
    width = canvas.winfo_reqwidth()
    canvas.create_text(width/2, 20, text=title, font="Sans 14")
    canvas.update()

In [2]:
######################################################################
# Question 1
######################################################################

In [3]:
dataset = pd.read_csv("football_teams.csv", header=None).values.tolist()

In [4]:
width, height, margin = 500, 500, 50
attributeX = [i[1] for i in dataset]
attributeY = [i[2] for i in dataset]
root = Tk()
canvas = Canvas(root, width=width, height=height, background="white")
canvas.pack()
canvas.data = {}
canvas.data["margin"] = margin
canvas.data["minX"], canvas.data["minY"], canvas.data["maxX"], canvas.data["maxY"] = min(attributeX), min(attributeY), max(attributeX), max(attributeY)

In [5]:
centroids1 = [(None, 4.0, 6.0),(None, 5.0, 4.0)]
centroids2 = [(None, 3.0, 3.0),(None, 8.0, 3.0)]
centroids3 = [(None, 2.0, 2.0),(None, 4.0, 8.0)]

In [6]:
clustering = {
    '1.1': kmeans(dataset, canvas, 2, False, 'Manhattan', centroids1), 
    '1.2': kmeans(dataset, canvas, 2, False, 'Euclidean', centroids1), 
    '1.3': kmeans(dataset, canvas, 2, False, 'Manhattan', centroids2), 
    '1.4': kmeans(dataset, canvas, 2, False, 'Manhattan', centroids3)
}

In [7]:
for i in clustering:
    print(i,'\nSimilarity: ', clustering[i]['similarity'],'\nSSE: ', clustering[i]['SSE'], '\nCentroids: ',clustering[i]['centroids'], '\n')

1.1 
Similarity:  Manhattan 
SSE:  54.095 
Centroids:  [(0, 4.0, 6.333333333333333), (1, 5.571428571428571, 3.5714285714285716)] 

1.2 
Similarity:  Euclidean 
SSE:  27.833 
Centroids:  [(0, 2.5, 5.0), (1, 6.833333333333333, 4.0)] 

1.3 
Similarity:  Manhattan 
SSE:  27.833 
Centroids:  [(0, 2.5, 5.0), (1, 6.833333333333333, 4.0)] 

1.4 
Similarity:  Manhattan 
SSE:  57.905 
Centroids:  [(0, 4.857142857142857, 3.5714285714285716), (1, 5.666666666666667, 6.333333333333333)] 



In [8]:
######################################################################
# Question 2
######################################################################

In [16]:
dataset = pd.read_csv('iris_adj.csv', header=None)

target = dataset[[0]]
features = dataset[[1,2,3,4]]
normalized_features = pd.DataFrame(normalize(features, axis=0))

dataset = pd.concat([target, normalized_features], axis=1)

dataset = dataset.values.tolist()

In [17]:
clustering = {'Euclidean': None, 'Cosine': None, 'Jaccard': None}
for i in clustering:
    clustering[i] = kmeans(dataset, canvas, 3, False, i)

In [12]:
######################################################################
# 2.1
######################################################################

In [18]:
for i in clustering:
    print('\nSimilarity: ', clustering[i]['similarity'],'\nSSE: ', clustering[i]['SSE'])


Similarity:  Euclidean 
SSE:  0.046

Similarity:  Cosine 
SSE:  0.056

Similarity:  Jaccard 
SSE:  0.05


In [14]:
######################################################################
# 2.2
######################################################################

In [19]:
for i in clustering:
    correct = 0
    for j in range(len(clustering[i]['clusters'])):
        targets = list(item[0] for item in clustering[i]['clusters'][j])
        #print(targets)
        modes = mode(targets)
        #print(modes)
        correct += targets.count(modes)
    print(i, 'Accuracy =',correct)

Euclidean Accuracy = 144
Cosine Accuracy = 142
Jaccard Accuracy = 150


In [20]:
######################################################################
# 2.3
######################################################################

In [21]:
for i in clustering:
    print(i,clustering[i]['iterations'])    

Euclidean 4
Cosine 7
Jaccard 3
